### Imports and parameters

In [1]:
import cv2 as cv
import copy as cp
import numpy as np
import numpy.random as rnd
import scipy.spatial.distance as scidst
from skimage.measure import compare_mse
from timeit import default_timer as timer

sources = [
    ["source_img.png",512,512], #0
    ["w3_tower.png",512,512],   #1
    ["lisa.jpg",512,512],       #2
    ["star_sky.jpg",512,512],   #3
    ["penguin.jpeg",512,512],   #4
    ["w3_ciri.png",1920,1080],   #5
    ["w3_ciri_small.png"],        #6
    ["target_star.png"],   #7
    ["btf_wing.jpg"]       #8
]
source = sources[8]
source_image_name = source[0]
source_image = cv.imread("./res/{}".format(source_image_name))
source_image_size = [source_image.shape[1],source_image.shape[0]]

print(source_image_size)

saving_interval = int(50000000/(source_image_size[0]*source_image_size[1]))

generation_size = 5
max_generation_cycles = 500
mutation_chance = 0.25
mutation_speed = 10.0
mutation_amount = 10
crossover_amount = 0.5 #percents
selection_percent = 0.8

#source_image_arr = np.array(source_image).reshape(512*512*4)
#source_image_norm = np.sum(source_image_arr)

[600, 800]


### Image Utils

In [2]:
def get_random_color():
    return (np.random.randint(256), np.random.randint(256),np.random.randint(256))#np.random.randint(256))#,transparency)

### Mutations

In [3]:
class CircleMutation:
    
    def __init__(self,x,y,radius,color):
        self.x, self.y = int(x), int(y)
        self.radius = int(radius)
        self.color = color
        
    def apply(self,genes):
        cv.circle(genes,(self.x,self.y),self.radius,self.color,thickness=cv.FILLED)
    
    @staticmethod
    def generate_random():
        w,h = source_image_size[0],source_image_size[1]
        x = np.random.randint(0,w)
        y = np.random.randint(0,h)
        #r = np.random.exponential(20,4)
        r = np.random.randint(0,8)
        gene = CircleMutation(x,y,r,get_random_color())
        return gene
    
    
class PolygonMutation:
    
    def __init__(self,points,color):
        self.points = points
        self.color = color
    
    def apply(self,genes):
        cv.fillPoly(genes,[self.points],self.color)
        
    @staticmethod
    def generate_random():
        vertices = 3
        w,h = source_image_size[0],source_image_size[1]
        rnd_point = [np.random.uniform(0,w),np.random.uniform(0,h)]
        size = [12,16]
        points = []
        for i in range(vertices):
            point_x = int(rnd_point[0]+np.random.uniform(-1,1)*size[0])
            point_y = int(rnd_point[1]+np.random.uniform(-1,1)*size[1])
            points.append([point_x,point_y])
        return PolygonMutation(np.array(points,dtype=np.int32),get_random_color())
    
    
class SquareMutation:
    
    def __init__(self,x,y,side,color):
        self.x, self.y = int(x), int(y)
        self.hside = int(side/2)
        self.color = color
        
    def apply(self,genes):
        start_point = (self.x-self.hside,self.y-self.hside)
        end_point = (self.x+self.hside,self.y+self.hside)
        cv.rectangle(genes,start_point,end_point,self.color,thickness=cv.FILLED)
    
    @staticmethod
    def generate_random():
        w,h = source_image_size[0],source_image_size[1]
        x = np.random.randint(0,w)
        y = np.random.randint(0,h)
        side = np.random.randint(0,8)
        gene = SquareMutation(x,y,side,get_random_color())
        return gene

### Chromosome

In [4]:
class Chromosome:
    
    def __init__(self,genes=None):
        if genes is None:
            genes = cv.imread("./out/latest_{}.png".format(source_image_name))
        if genes is None:
            genes = np.zeros_like(source_image)
        self.genes = genes
        self.fitness = Chromosome.compute_fitness(self.genes)
    
    def mutate(self):
        for i in range(4):
            t0 = timer()
            choice = np.random.randint(0,3)
            mutation = None
            if choice==0:
                mutation = CircleMutation.generate_random()
            if choice==1:
                mutation = SquareMutation.generate_random()
            if choice==2:
                mutation = PolygonMutation.generate_random()
            new_genes = self.genes.copy()
            mutation.apply(new_genes)
            new_fitness = Chromosome.compute_fitness(new_genes)
            if new_fitness > self.fitness:
                self.genes = new_genes
                self.fitness = new_fitness
    
    @staticmethod
    def cross(chr1,chr2):
        w_sum = chr1.fitness + chr2.fitness
        w1, w2 = chr1.fitness/w_sum, chr2.fitness/ w_sum
        result = cv.addWeighted(chr1.genes,w1,chr2.genes,w2,0)
        #result = cv.addWeighted(chr1.genes,0.5,chr2.genes,0.5,0)
        return Chromosome(result)
    
    @staticmethod
    def compute_fitness(genes,target=source_image):
        return 1/(1 + compare_mse(source_image,genes))
    
    def __lt__(self, other):
        return self.fitness < other.fitness
            
    def __eq__(self, other):
        return self.fitness == other.fitness
    
    
p_mutate = np.vectorize(Chromosome.mutate)

### GA Engine

In [5]:
def do_mutate(generation):
    p_mutate(generation)
    #for i in range(len(generation)):
     #   generation[i].mutate()
      
    
def do_crossover(generation,crossover_percent=0.7):
    limit = int(np.ceil(crossover_percent*len(generation)))
    result_generation = []
    for i in range(1,limit):
        result_generation.append(Chromosome.cross(generation[i-1],generation[i]))
    return result_generation


def do_select(generation,select_percent=0.8):
    limit = int(np.ceil(select_percent*len(generation)))
    generation[::-1].sort()
    return generation[:limit]


def do_simulate(population_size = 40,population_limit=200000):
    generation = np.array([Chromosome() for i in range(population_size)])
    best_image = generation[0].genes
    best_fitness = generation[0].fitness
    for i in range(population_limit):
        t1 = timer()
        new_generation = do_crossover(generation,0.6)
        t2 = timer()
        generation = np.concatenate((generation,new_generation))
        t3 = timer()
        do_mutate(generation)
        t4 = timer()
        generation = do_select(generation,0.8)
        t5 = timer()
        #rint("Crossover     time : {}".format(t2-t1)), print("Concatenation time : {}".format(t3-t2)), print("Mutation      time : {}".format(t4-t3)), int("Selection     time : {}".format(t5-t4))
        if generation[0].fitness > best_fitness:
            best_fitness = generation[0].fitness
            best_image = generation[0].genes
        if len(generation) > population_size:
            generation = generation[:population_size]
        print("Generation {} : best fitness {}".format(i,best_fitness))
        #if i % 10 == 0:
          #  cv.imwrite("./out/result{}.png".format(i),best_image)
        if i % saving_interval == 0:
            cv.imwrite("./out/latest_{}.png".format(source_image_name),best_image)
            print("Saved")

### Main

In [6]:
def main():
    try:
        chromosome = do_simulate(population_size=3)
        return chromosome.image
    except KeyboardInterrupt:
        return None
    
main()

Generation 0 : best fitness 0.000312154046065568
Saved
Generation 1 : best fitness 0.0003121541040560005
Generation 2 : best fitness 0.0003121676683867728
